In [25]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

In [26]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_cortadas"

Obtenemos de las carpetas originales las fotos

In [27]:
from PCA_funciones import matriz_fotos_desde_carpeta


#obtiene imagenes vectorizadas 30*30 y el nombre de la persona de cada foto
image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas) 

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


## Aplicamos Backpropagation
Transformamos las imagenes, con la cantidad de copoenentes 

### Escalamos y reducimos con PCA las imagenes

In [28]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" 
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

from PCA_funciones import transformar_imagen_pca

"""# Función para transformar una imagen con un rango específico de componentes PCA
def transformar_imagen_pca(imagen, scaler, pca_components, num_componentes=50, inicio_componente=0):
    # Estandarizar la imagen
    imagen_estandarizada = scaler.transform(imagen)
    
    # Seleccionar el rango de componentes
    componentes_seleccionados = pca_components[inicio_componente:inicio_componente + num_componentes]
    
    # Realizar la proyección manualmente
    imagen_pca = np.dot(imagen_estandarizada, componentes_seleccionados.T)
    return imagen_pca
"""

# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)


[[ 7.22160860e+00 -7.80849051e+00 -2.07612021e-01 ... -7.16897218e-01
  -2.28025618e-01 -3.38966655e-01]
 [-7.84286624e+00 -1.12024841e+01  4.89598300e-01 ... -7.43509551e-01
  -6.18654161e-01  2.88035939e-01]
 [-6.55630242e+00 -7.80088794e+00 -1.49179444e+00 ... -1.37140978e+00
  -2.96450608e-01  2.05965925e-02]
 ...
 [ 3.55657307e+00  6.49301206e-01 -6.23891347e+00 ... -6.32500626e-02
   3.18870940e-01 -6.26304974e-01]
 [-2.65755973e+00 -4.44534213e-03 -8.82886566e+00 ...  2.92722209e-01
  -9.78650328e-02 -2.65089331e-01]
 [-2.62864833e+00  8.66772857e-01 -8.11920552e+00 ...  1.20877517e-01
  -1.40180893e-02 -1.42284246e-01]]


In [30]:
print("shape reducido " , imagenes_pca_reducidas.shape) # la segunda cantidad indica el numero de componentes una vez reducido
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print("Cantidad de Personas: " , cantidad_personas) 
cantidad_fotos = len(image_person)
print("Cantidad Total de Fotos: " , cantidad_fotos)


shape reducido  (1206, 60)
Cantidad de Personas:  19
Cantidad Total de Fotos:  1206


## Back Propagation

### Transformamos los nombres de las personas a vector de salida

In [31]:
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
print(salida_lista)
salida_lista.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(1206, 19)

### Inicializamos parametros

In [32]:
# el vector de entrada X
X=imagenes_pca_reducidas
X.shape

X = imagenes_pca_reducidas
Y = salida_lista

# Convierte la lista a un arreglo NumPy
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 30

Separamos conjunto de datos de entrenamiento y test

In [33]:

# Semilla para reproducibilidad
np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

## Back propagation manual

In [18]:


# 
# lo hago con  matrices de pesos
# puedo tener tantos inputs como quiera
# puedo tener tantas neuronas ocultas como quiera
# puedo tener tanas neuronas de salida como quiera
# fuera de este codigo esta la decision que tomo segun el valor de salida de cada neurona de salida

import math
import numpy as np
import matplotlib.pyplot as plt

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
            
        case "softmax":
            exps = np.exp(x - np.max(x))  # Stabilization trick
            y = exps / np.sum(exps)            
    return y

func_eval_vec = np.vectorize(func_eval)

def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
        case "softmax":
            d = y * (1.0 - y)  #La derivada de softmax es más compleja y normalmente se calcula usando la matriz Jacobiana. 
                                # En este ejemplo, hemos usado una simplificación y * (1.0 - y) que es una aproximación y 
                                # puede no ser precisa en todos los casos.            
    return d

deriv_eval_vec = np.vectorize(deriv_eval)


In [34]:
print(len(X))
print(len(X_train))
print(len(X_test))

1206
964
242


In [35]:
#new_faces_array_transformed.shape #vemos el vector con las caras y sus 60 componentes a partir de la 3ra

# Normalización de las entradas
#mean = np.mean(X, axis=0)
#std = np.std(X, axis=0)
#X = (X - mean) / std

# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
#salida = np.array(salida_lista)
#Y = salida #salida.reshape(len(X),1)

filas_qty = len(X_train)        # la cantidad de fotos
input_size = X_train.shape[1]   # entradas la cantidad de componentes
                                # hidden_size = 3  # neuronas capa oculta
hidden_size1 = filas_qty        # Neuronas en la primera capa oculta, arrancamos probando con la cantidad de fotos
hidden_size2 = 3                # Neuronas en la segunda capa oculta

output_size = cantidad_personas # vector con la probabilidad q sea la persona de la posicion

# defino las funciones de activacion de cada capa
hidden_FUNC = 'logsig'  # probamos con  la logistica para las capas ocultas
output_FUNC = 'tansig'  # probamos la tangente hiperbolica para la capa de salida

# Incializo las matrices de pesos azarosamente
# W1 son los pesos que van del input a la capa oculta
# W2 son los pesos que van de la capa oculta a la segunda capa oculta
# W3 son los pesos que van de la 2da Capa Oculta a la salida

np.random.seed(1021) # random seed para que las corridas sean reproducibles
W1 = np.random.uniform(-0.5, 0.5, [hidden_size1, input_size])
X01 = np.random.uniform(-0.5, 0.5, [hidden_size1, 1] )
W2 = np.random.uniform(-0.5, 0.5, [hidden_size2, hidden_size1])
X02 = np.random.uniform(-0.5, 0.5, [hidden_size2, 1] )
W3 = np.random.uniform(-0.5, 0.5, [output_size, hidden_size2])
X03 = np.random.uniform(-0.5, 0.5, [output_size, 1] )

#Aca estamos trabajando 

In [68]:
# # Avanzo la red, forward
# # para TODOS los X al mismo tiempo ! 
# #  @ hace el producto de una matrix por un vector_columna
# hidden_estimulos = W1 @ X.T + X01
# hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
# output_estimulos = W2 @ hidden_salidas + X02
# output_salidas = func_eval_vec(output_FUNC, output_estimulos)


# Función de propagación hacia adelante (forward propagation)
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
    hidden1_estimulos = W1 @ X.T + b1
    hidden1_salidas = func_eval_vec(hidden_FUNC, hidden1_estimulos)
    hidden2_estimulos = W2 @ hidden1_salidas + b2
    hidden2_salidas = func_eval_vec(hidden_FUNC, hidden2_estimulos)
    output_estimulos = W3 @ hidden2_salidas + b3
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    return hidden1_salidas, hidden2_salidas, output_salidas, hidden1_estimulos, hidden2_estimulos, output_estimulos


hidden_salida1, hidden_salida2, output_salidas, _, _ ,_ = forward_propagation(X_train, W1, X01, W2, X02, W3, X03)


# calculo el error promedi general de TODOS los X
Error= np.mean( (Y_train.T - output_salidas)**2 )

# Inicializo
#epoch_limit = 5000    # para terminar si no converge
Error_umbral = 1.0e-06
#Error_umbral = 1.0e-15
learning_rate = 0.5
#learning_rate = 0.2
Error_last = 10    # lo debo poner algo dist a 0 la primera vez
epoch = 0

while ( math.fabs(Error_last-Error)>Error_umbral and (epoch < epoch_limit)):
    epoch += 1
    Error_last = Error

    # recorro siempre TODA la entrada
    for fila in range(filas_qty): #para cada input x_sub_fila del vector X
        # propagar el x hacia adelante
        # hidden_estimulos = W1 @ X[fila:fila+1, :].T + X01
        # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
        # output_estimulos = W2 @ hidden_salidas + X02
        # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
        hidden_salida1, hidden_salida2, output_salidas,  _, _ ,_  = forward_propagation(X_train[fila:fila+1, :], W1, X01, W2, X02, W3, X03)

        # calculo los errores en la capa hidden y la capa output
        ErrorSalida = Y_train[fila:fila+1,:].T - output_salidas
        # output_delta es un solo numero
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        
        # hidden_delta es un vector columna
        # hidden_delta = deriv_eval_vec(hidden_FUNC, hidden_salidas)*(W2.T @ output_delta)
        # hidden_delta es un vector columna
        hidden_delta2 = deriv_eval_vec(hidden_FUNC, hidden_salida2)*(W3.T @ output_delta)

        # hidden_delta es un vector columna
        hidden_delta1 = deriv_eval_vec(hidden_FUNC, hidden_salida1)*(W2.T @ hidden_delta2)

        # ya tengo los errores que comete cada capa
        # corregir matrices de pesos, voy hacia atras
        # backpropagation
        W1 = W1 + learning_rate * (hidden_delta1 @ X_train[fila:fila+1, :] )
        X01 = X01 + learning_rate * hidden_delta1
        W2 = W2 + learning_rate * (hidden_delta2 @ hidden_salida1.T )
        X02 = X02 + learning_rate * hidden_delta2
        W3 = W3 + learning_rate * (output_delta @ hidden_salida2.T)
        X03 = X03 + learning_rate * output_delta

    # ya recalcule las matrices de pesos
    # ahora avanzo la red, feed-forward
    # hidden_estimulos = W1 @ X.T + X01
    # hidden_salidas = func_eval_vec(hidden_FUNC, hidden_estimulos)
    # output_estimulos = W2 @ hidden_salidas + X02
    # output_salidas = func_eval_vec(output_FUNC, output_estimulos)
    
    hidden_salida1, hidden_salida2, output_salidas,  hidden_estimulo1, hidden_estimulo2 , hidden_ouput  = forward_propagation(X_train[fila:fila+1, :], W1, X01, W2, X02, W3, X03)
    

    # calculo el error promedio general de TODOS los X
    Error= np.mean( (Y_train.T - output_salidas)**2 )

    # tengo que hacer X01.T[0]  para que pase el vector
   # grafico.graficarVarias(W1, X01.T[0], epoch, -1)


# Evaluar el modelo en el conjunto de prueba
# loss, accuracy = model.evaluate(X_test, y_test)
# print("Accuracy on test set:", accuracy)

# Hacer predicciones con el modelo entrenado
# predictions = model.predict(new_data)
Error

0.06790324408870003

In [69]:
hidden_salida1_modelo, hidden_salida2_modelo, output_salidas_modelo,  hidden_estimulo1_modelo, hidden_estimulo2_modelo , hidden_ouput_modelo  = forward_propagation(X_train, W1, X01, W2, X02, W3, X03)
    


In [70]:
print(output_salidas_modelo.shape)

salidas_train = output_salidas_modelo.T
salidas_train

(19, 964)


array([[-1.88872153e-04,  1.59390445e-05,  3.23671492e-01, ...,
         1.86354635e-04,  7.01049876e-06,  3.05646399e-05],
       [-1.89811393e-04,  1.60032534e-05,  3.23671496e-01, ...,
         1.86361344e-04,  7.03808016e-06,  3.06836140e-05],
       [-1.89810607e-04,  1.60028726e-05,  3.23671496e-01, ...,
         1.86360605e-04,  7.03836836e-06,  3.06835706e-05],
       ...,
       [-1.88734676e-04,  1.59295507e-05,  3.23671492e-01, ...,
         1.86353441e-04,  7.00655048e-06,  3.05472493e-05],
       [-1.89813633e-04,  1.60043411e-05,  3.23671494e-01, ...,
         1.86363452e-04,  7.03726008e-06,  3.06837245e-05],
       [-1.72769548e-04,  1.48415765e-05,  3.23671424e-01, ...,
         1.86247169e-04,  6.53441391e-06,  2.85244531e-05]])

In [71]:
sum(salidas_train[1])

1.219444648765915

In [65]:
output_salidas

array([[-2.46149047e-05],
       [ 3.70818546e-06],
       [ 3.23669067e-01],
       [ 8.37281855e-03],
       [ 2.70774081e-06],
       [ 1.97290873e-07],
       [ 1.18310036e-02],
       [ 4.21148413e-03],
       [ 7.24846701e-07],
       [ 4.99732695e-06],
       [ 9.95954899e-02],
       [ 1.48364915e-03],
       [ 2.11091124e-06],
       [ 4.62118395e-01],
       [ 3.07988615e-01],
       [ 6.81040907e-07],
       [ 1.84834471e-04],
       [ 1.19062199e-06],
       [ 1.29090719e-05]])

### Calcular el modelo sobre test

## Nuevas Fotos

In [23]:
p_Fotos = root_path + "Nuevas_Fotos"
p_Fotos

'../Nuevas_Fotos'

In [39]:
from Funciones import  Leer_fotos
fotos=Leer_fotos(p_Fotos)
fotos

{'Abel': [array([[[201, 206, 215],
          [202, 207, 216],
          [203, 208, 217],
          ...,
          [105, 131, 167],
          [105, 131, 167],
          [105, 131, 167]],
  
         [[165, 174, 187],
          [164, 176, 188],
          [167, 176, 189],
          ...,
          [106, 132, 168],
          [106, 132, 168],
          [106, 132, 168]],
  
         [[116, 137, 158],
          [115, 139, 159],
          [117, 138, 159],
          ...,
          [107, 133, 169],
          [108, 134, 170],
          [108, 134, 170]],
  
         ...,
  
         [[103,  73,  14],
          [ 93,  64,   3],
          [ 87,  57,   0],
          ...,
          [ 52,  35,   2],
          [ 54,  37,   4],
          [ 57,  38,   3]],
  
         [[ 94,  66,   5],
          [ 91,  64,   0],
          [ 90,  62,   1],
          ...,
          [ 52,  33,   0],
          [ 55,  35,   0],
          [ 58,  39,   2]],
  
         [[ 94,  67,   3],
          [ 92,  65,   1],
          [ 91, 

In [12]:
# Mostrar información
nombres_personas = list(fotos.keys())
print("Cantidad de personas:", len(nombres_personas))
print("Cantidad de fotos por persona:")
for nombre_persona, fotos_persona in fotos.items():
    print(f"- {nombre_persona}: {len(fotos_persona)}")

Cantidad de personas: 15
Cantidad de fotos por persona:
- Abel: 1
- Carlos: 1
- Federico G: 1
- Federico R: 1
- Franco S: 1
- Gerard: 1
- Gustavo: 1
- Joaquin: 1
- Lautaro: 1
- Lisandro: 1
- Matias: 1
- Natalia: 1
- Noelia: 1
- Paola: 1
- Victorio: 1


In [28]:
def recortar_imagen_C(image):
    import os
    import cv2
   # import matplotlib.pyplot as plt
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

In [15]:
import os
import cv2
#import matplotlib.pyplot as plt

def recortar_imagen(image):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

In [18]:
def procesar_carpetas(directorio_origen, directorio_destino):
    # Iterar sobre las carpetas dentro del directorio

    # Crea el directorio si no existe
    if not os.path.exists(directorio_destino):
       os.makedirs(directorio_destino)

    for root, dirs, files in os.walk(directorio_origen):
        for dir_name in dirs:
            print("Carpeta:", dir_name)
            dir_path = os.path.join(root, dir_name) #directorio origen de la persona 
            dir_path_recorte = os.path.join(directorio_destino, dir_name) #directorio destino de la persona
            
            if not os.path.exists(dir_path_recorte):
                os.makedirs(dir_path_recorte) #creamos carpetas para cada nombre
            
            images = []

            numero=0 #numero de la foto dentro del directorio
            
            for file in glob.glob(os.path.join(dir_path, "*.jp*g")) :
                # Verificar si el archivo es una imagen
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    # Leer la imagen
                    #mage_path = os.path.join(dir_path, file)
                    image_path = file
                    print(image_path)
                    image = cv2.imread(image_path)
                    
                    if image is not None:
                        # Recortar la imagen
                        numero +=1
                        face_images = recortar_imagen(image)
                        images.extend(face_images)
                        numero_interno=0
                        for face_image in face_images:
                            gray_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                            numero_interno +=1
                            numero_formateado = str(numero).zfill(2)
                            numero_interno_formateado = str(numero_interno).zfill(2)
                            #_{image_path.split('.')[0]}_
                            output_path = os.path.join(dir_path_recorte , f"{dir_name}_{numero_formateado}_{numero_interno_formateado}.jpg")
                            # Save the cropped face to the output directory                       
                            cv2.imwrite(output_path, gray_image)



In [19]:
nuevas_recortadas = root_path + "nuevas_recortadas"
procesar_carpetas(p_Fotos, nuevas_recortadas)

Carpeta: Abel
../Nuevas_Fotos\Abel\Abel.jpeg
Carpeta: Carlos
../Nuevas_Fotos\Carlos\Carlos.jpeg
Carpeta: Federico G
../Nuevas_Fotos\Federico G\Federico G.jpeg
Carpeta: Federico R
../Nuevas_Fotos\Federico R\Federico R.jpeg
Carpeta: Franco S
../Nuevas_Fotos\Franco S\Franco S.jpeg
Carpeta: Gerard
../Nuevas_Fotos\Gerard\Gerard.jpeg
Carpeta: Gustavo
../Nuevas_Fotos\Gustavo\Gustavo.jpeg
Carpeta: Joaquin
../Nuevas_Fotos\Joaquin\Joaquin.jpeg
Carpeta: Lautaro
../Nuevas_Fotos\Lautaro\Lautaro.jpeg
Carpeta: Lisandro
../Nuevas_Fotos\Lisandro\Lisandro.jpeg
Carpeta: Matias
../Nuevas_Fotos\Matias\Matias.jpeg
Carpeta: Natalia
../Nuevas_Fotos\Natalia\Natalia.jpeg
Carpeta: Noelia
../Nuevas_Fotos\Noelia\noelia.jpeg
Carpeta: Paola
../Nuevas_Fotos\Paola\Paola.jpeg
Carpeta: Victorio
../Nuevas_Fotos\Victorio\Victorio.jpeg


In [20]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(nuevas_recortadas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


In [ ]:
num_componentes=60
inicio_componente
# Transformar la nueva imagen utilizando los componentes especificados
imagen_pca_reducida = transformar_imagen_pca(image_matrix, scaler, pca_components, num_componentes, inicio_componente)

imagen_pca_reducida.shape

In [ ]:


prediction = model.predict(imagen_pca_reducida)

print("Predicción:", prediction)